# JEM092 Asset Pricing
# Seminar 11
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 28.4.2022

In [ ]:
# Import third-party packages.
library(lubridate)
library(quantmod)
library(sandwich)
library(xts)

# Data preparation

In [ ]:
load('Asset_Pricing_seminar_data.RData')
ls() 

In [ ]:
START_DATE <- '2005-01-01'
END_DATE <- '2021-03-31'

# Load stock specific data.
prices <- c()
bm_ratios <- c()
market_caps <- c()
for (i in 1:length(OHLCV_sap100)) {
    ticker <- as.character(names(OHLCV_sap100)[i])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_prices = stock_data[, 6]
    stock_bm_ratios <- book_value_sap100[[i]][, 2] / book_value_sap100[[i]][, 1]
    stock_market_caps <- MktCap_sap100[[i]]
    colnames(stock_prices) <- ticker
    colnames(stock_bm_ratios) <- ticker
    colnames(stock_market_caps) <- ticker
    prices <- cbind(prices, stock_prices)
    bm_ratios <- cbind(bm_ratios, stock_bm_ratios)
    market_caps <- cbind(market_caps, stock_market_caps)
}

# Forward-fill book-to-market ratios (observed quarterly) and drop the last row.
bm_ratios <- na.locf(bm_ratios)
bm_ratios <- head(bm_ratios, -1)

# Load daily FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip",
    destfile = "F-F_Research_Data_Factors_daily.zip"
)
unzip("F-F_Research_Data_Factors_daily.zip")
ff3_factors <- read.delim(
    'F-F_Research_Data_Factors_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 24957,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
ff3_factors[['t']] <- as.Date(as.character(ff3_factors[['t']]), '%Y%m%d')
ff3_factors <- as.xts(ff3_factors[, 2:5], order.by = ff3_factors[['t']])
ff3_factors <- ff3_factors / 100

# Load monthly FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
unzip("F-F_Research_Data_Factors.zip")
monthly_ff3_factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1137,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
monthly_ff3_factors[['t']] <- as.Date(paste0(as.character(monthly_ff3_factors[['t']]), '01'), '%Y%m%d')
monthly_ff3_factors <- as.xts(monthly_ff3_factors[, 2:5], order.by = monthly_ff3_factors[['t']])
monthly_ff3_factors <- monthly_ff3_factors / 100
index(monthly_ff3_factors) <- as.Date(as.yearmon(index(monthly_ff3_factors)), frac = 1)

In [ ]:
prices[1:5, 1:3]
market_caps[201:205, 1:3]
bm_ratios[20:25, 1:3]
ff3_factors[1:5,]
monthly_ff3_factors[1:5,]

In [ ]:
# Compute daily and monthly excess returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
monthly_returns <- lapply(prices, monthlyReturn, USE.NAMES = TRUE)
daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <- colnames(prices)
colnames(monthly_returns) <- colnames(prices)
index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns)), frac = 1)
for (i in 1:ncol(monthly_returns)) {
    daily_returns[, i] <- daily_returns[, i] - ff3_factors[, 4]
    monthly_returns[, i] <- monthly_returns[, i] - monthly_ff3_factors[, 4]
}

# Compute monthly market caps and sizes.
sizes <- log(market_caps)
monthly_market_caps <- apply.monthly(market_caps, tail, 1)
monthly_sizes <- apply.monthly(sizes, tail, 1)
# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))
index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
index(monthly_sizes) <- as.Date(as.yearmon(index(monthly_sizes)), frac = 1)

In [ ]:
daily_returns[1:5, 1:3]
monthly_returns[1:5, 1:3]
monthly_market_caps[1:5, 1:3]
monthly_sizes[1:5, 1:3]

## Compute book-to-market ratios

In [ ]:
# Initialize an xts object for storing book-to-market ratios.
monthly_bm_ratios <- as.xts(
    matrix(
        nrow = nrow(bm_ratios),
        ncol = ncol(bm_ratios)
    ),
    order.by = index(bm_ratios)
)
names(monthly_bm_ratios) <- names(bm_ratios)

start_year <- as.numeric(format(min(index(monthly_bm_ratios)), '%Y'))
end_year <- as.numeric(format(max(index(monthly_bm_ratios)), '%Y'))

# Iterate over rows and compute BM ratios.
for (i in (start_year + 2):end_year) {
    start_day <- as.Date(paste0(i - 1, '06', '30'), format = '%Y%m%d')
    end_day <- as.Date(paste0(i, '05', '31'), format = '%Y%m%d')
    dates <- paste0(start_day, '/', end_day)
    end_of_previous_year <- as.Date(paste0(i - 2, '12', '31'), format = '%Y%m%d')
    for (j in 1:ncol(bm_ratios)) {
        monthly_bm_ratios[dates, j] <- as.numeric(bm_ratios[end_of_previous_year, j][[1]])
    }
}

In [ ]:
monthly_bm_ratios[41:55, 1:3]
bm_ratios[31:45, 1:3]

## Compute momentum

In [ ]:
# Initialize an xts object for storing momentum.
monthly_momentum <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_momentum) <- names(monthly_returns)

# Iterate over rows and compute momentum.
for (i in 2:nrow(monthly_momentum)) {
    current_day <- index(monthly_momentum[i])
    end_day <- index(monthly_momentum[i - 1])
    start_day <- end_day %m-% months(10)
    dates <- paste0(start_day, '/', end_day)
    returns <- monthly_returns[dates]

    if (nrow(returns) < 11) {
        next
    }

    for (j in 1:ncol(monthly_momentum)) {
        monthly_momentum[current_day, j] <- 100 * (prod(returns[, j] + 1) - 1)
    }
}

In [ ]:
monthly_momentum[11:15, 1:3]

## Compute short-term reversal

In [ ]:
# Initialize an xts object for storing reversal
monthly_reversal <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_reversal) <- names(monthly_returns)

# Iterate over rows and compute reversal.
for (i in 1:nrow(monthly_reversal)) {
    monthly_reversal[i,] <- monthly_returns[i] * 100
}

In [ ]:
monthly_reversal[11:15, 1:3]

## Estimate betas

Use the following regression equation:

$r_{i,t} = \alpha_i + \beta_i MKT_t + \epsilon_{i,t}$

- $i$ denotes stocks
- $t$ denotes time
- $r$ is the excess stock return
- $MKT$ is the excess market return
- $\alpha$ is the intercept
- $\beta$ is the slope coefficient
- $\epsilon$ is the error term

Let's illustrate the estimation on daily data for 5 different periods: 1, 3, 6, 12, and 24 months.

In [ ]:
# Initialize an xts object for storing betas.
betas_period <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(betas_period) <- names(monthly_returns)
betas <- list(betas_period, betas_period, betas_period, betas_period, betas_period)

# Iterate over rows and compute betas.
for (i in 1:nrow(betas_period)) {
    end_day <- index(betas_period[i])

    for (p_b in list(c(1, 1), c(3, 2), c(6, 3), c(12, 4), c(24, 5))) {
        start_day <- end_day %m-% months(p_b[[1]])
        dates <- paste0(start_day, '/', end_day)
        returns <- daily_returns[dates]
        market_returns <- ff3_factors[dates]

        # Estimate betas for each stock.
        for (j in 1:ncol(betas_period)) {
            stock_returns <- returns[, j]
            # Merge excess returns and market excess returns.
            model_data <- cbind(stock_returns, market_returns[, 1])
            model_data <- na.omit(model_data)
            # Don't estimate betas if the number of observations is lower than 200.
            if (nrow(model_data) < p_b[[1]] * 15) {
                next
            }
            model_betas <- lm(model_data[, 1] ~ model_data[, 2])
            # Save the given beta to `betas`.
            betas[[p_b[[2]]]][i, j] <- model_betas$coefficients[[2]]
        }
    }
}

monthly_betas_1m <- betas[[1]]
monthly_betas_3m <- betas[[2]]
monthly_betas_6m <- betas[[3]]
monthly_betas_12m <- betas[[4]]
monthly_betas_24m <- betas[[5]]

In [ ]:
monthly_betas_1m[1:5, 1:3]
monthly_betas_3m[1:5, 1:3]
monthly_betas_6m[11:15, 1:3]
monthly_betas_12m[21:25, 1:3]
monthly_betas_24m[21:25, 1:3]

# Define functions for portfolio sorts and Fama-MacBeth regressions

## Univariate portfolio sorts

In [ ]:
perform_univariate_portfolio_sort <- function (sort_variable_data, returns, market_caps, n) {
    diff_portfolio <- paste(n, '- 1')

    # Initialize an xts object for storing portfolio returns.
    portfolio_returns <- as.xts(
        matrix(
            nrow = nrow(sort_variable_data) - 1,
            ncol = n + 1
        ),
        order.by = index(sort_variable_data[2:nrow(sort_variable_data)])
    )
    names(portfolio_returns) <- c(1:n, diff_portfolio)

    # Iterate over rows, find breakpoints and compute monthly returns within the given value breakpoints.
    for (i in 1:nrow(portfolio_returns)) {
        current_month <- index(portfolio_returns[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(returns)) {
            next
        }

        # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
        month_returns <- returns[next_month]
        month_bm_ratios <- sort_variable_data[current_month]
        month_market_caps <- market_caps[next_month]
        # Replace NA market caps with 0s so that such observations have no weights.
        month_market_caps[is.na(month_market_caps)] <- 0

        breakpoints <- quantile(month_bm_ratios, 0:n/n, na.rm = TRUE)
        not_na <- !is.na(month_bm_ratios)

        for (j in 1:n) {
            filter <- (breakpoints[[j]] < month_bm_ratios) & (month_bm_ratios < breakpoints[[j + 1]]) & not_na
            # Compute weighted average portfolio returns.
            portfolio_returns[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
        }
        portfolio_returns[i, diff_portfolio] <- portfolio_returns[i, n] - portfolio_returns[i, 1]
    }

    # Compute overall average returns within portfolios and their standard errors.
    results_bm_ratios <- as.data.frame(matrix(nrow = 2, ncol = n + 1))
    names(results_bm_ratios) <- c(1:n, diff_portfolio)

    for (i in 1:ncol(results_bm_ratios)) {
        model <- lm(na.omit(portfolio_returns[, i]) ~ 1)
        results_bm_ratios[1, i] <- model$coefficients[[1]]
        results_bm_ratios[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
    }

    results_bm_ratios
}

## Fama-MacBeth regressions

In [ ]:
perform_fama_macbeth_regression <- function (data_explained, data_explanatory) {
    # Initialize an xts object for storing regression results.
    fm_results <- as.xts(
        matrix(
            nrow = nrow(data_explained) - 1,
            ncol = 1 + length(data_explanatory)
        ),
        order.by = index(data_explained[2:nrow(data_explained)])
    )
    column_names <- c('Intercept')
    column_names <- c(column_names, names(data_explanatory))
    names(fm_results) <- column_names

    # Iterate over rows and perform cross-sectional regressions.
    for (i in 1:nrow(fm_results)) {
        current_month <- index(fm_results[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(data_explained)) {
            next
        }

        # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
        month_data_explained <- data_explained[next_month]
        month_data_explanatory <- c()
        for (j in 1:length(data_explanatory)) {
            if (!current_month %in% index(data_explanatory[[j]])) {
                move_to_next_month <- TRUE
                break
            }
            move_to_next_month <- ncol(data_explanatory[[j]][current_month]) == sum(is.na(data_explanatory[[j]][current_month]))
            if (move_to_next_month) {
                break
            }
            colnames(month_data_explanatory)
            cross_section <- t(data_explanatory[[j]][current_month])
            colnames(cross_section) <- names(data_explanatory)[j]

            cross_section[which(cross_section == -Inf)] = NA
            cross_section[which(cross_section == Inf)] = NA

            month_data_explanatory <- cbind(month_data_explanatory, cross_section)
        }

        if (move_to_next_month) {
            next
        }

        transposed_explained <- t(month_data_explained)
        colnames(transposed_explained) <- c('explained')
        equation <- paste('explained ~ 1 + ', paste0(colnames(month_data_explanatory), collapse = ' + '))
        all_data <- na.omit(as.data.frame(cbind(transposed_explained, month_data_explanatory)))

        # Save regression coefficients and other statistics.
        model <- lm(equation, data = all_data)
        for (k in 1:ncol(fm_results)) {
            fm_results[i, k] <- model$coefficients[[k]]
        }
    }

    # Compute time series means, standard errors.
    final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = ncol(fm_results)))
    names(final_fm_results) <- colnames(fm_results)

    for (i in 1:ncol(final_fm_results)) {
        model <- lm(na.omit(fm_results[, i]) ~ 1)
        final_fm_results[1, i] <- model$coefficients[[1]]
        final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
    }

    final_fm_results
}

# Value

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
perform_univariate_portfolio_sort(monthly_bm_ratios, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regression analysis

In [ ]:
data <- list(monthly_bm_ratios)
names(data) <- c('Value')
perform_fama_macbeth_regression(monthly_returns, data)

## Fama-MacBeth regressions of returns on BM ratios, betas and sizes

In [ ]:
data <- list(monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)

# Momentum

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
perform_univariate_portfolio_sort(monthly_momentum, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regression analysis

In [ ]:
data <- list(monthly_momentum)
names(data) <- c('Momentum')
perform_fama_macbeth_regression(monthly_returns, data)

## Fama-MacBeth regressions of returns on momentum, BM ratios, betas and sizes

In [ ]:
data <- list(monthly_momentum, monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Momentum', 'Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)

# Short-term Reversal

## Portfolio analysis using monthly univariate decile portfolios

### Equally weighted returns

In [ ]:
monthly_equal_weights <- monthly_market_caps
monthly_equal_weights[,] <- 1
perform_univariate_portfolio_sort(monthly_reversal, monthly_returns, monthly_equal_weights, 10)

### Weighted returns

In [ ]:
perform_univariate_portfolio_sort(monthly_reversal, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regressions of returns on reversal, momentum, BM ratios, betas and sizes

In [ ]:
data <- list(monthly_reversal, monthly_momentum, monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Reversal', 'Momentum', 'Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)

In [ ]:
save(
    daily_returns,
    monthly_betas_24m,
    monthly_bm_ratios,
    monthly_ff3_factors,
    monthly_market_caps,
    monthly_momentum,
    monthly_returns,
    monthly_reversal,
    monthly_sizes,
    file = 'seminar_11.RData'
)